In [1]:
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from datetime import datetime
import time

# Headers required for stats.nba.com API 
HEADERS = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.nba.com/',
    'Origin': 'https://www.nba.com',
    'Connection': 'keep-alive',
}

In [2]:
def generate_ordered_constants(n_eff):
    """
    Generate (a1,b1,c1,d1, a2,b2,c2,d2) based on adjusted win-rate n_eff ∈ [0,1].
    """
    a2 = (1 - n_eff) * 10 + np.random.uniform(0, 1)
    b2 = (1 - n_eff) * 8  + np.random.uniform(0, 1)
    c2 = (1 - n_eff) * 5  + np.random.uniform(0, 1)
    d2 = (1 - n_eff) * 2  + np.random.uniform(0, 1)

    a1 = a2 + np.random.uniform(1, 2)
    b1 = b2 - np.random.uniform(1, 2)
    c1 = c2 + np.random.uniform(1, 2)
    d1 = c1 + np.random.uniform(1, 2)

    return a1, b1, c1, d1, a2, b2, c2, d2


In [3]:
class Team:
    def __init__(self, name, wins, games_played):
        self.name = name
        self.wins = wins
        self.games_played = games_played
        self.n = wins / games_played

    def payoff_matrix(self, n_eff):
        """
        Returns two 2×2 numpy arrays: (playoff_matrix, draft_matrix) generated using adjusted coefficient n_eff.
        """
        a1, b1, c1, d1, a2, b2, c2, d2 = generate_ordered_constants(n_eff)
        pm = np.array([[a1, b1],
                       [c1, d1]])
        dm = np.array([[a2, b2],
                       [c2, d2]])
        return pm, dm

    def expected_payoff(self, n_eff):
        """
        Returns the 2×2 expected payoff matrix with adjusted n_eff: M_exp = n_eff * pm + (1 - n_eff) * dm.
        """
        pm, dm = self.payoff_matrix(n_eff)
        return n_eff * pm + (1 - n_eff) * dm


In [4]:
def tank_GT(team: Team):
    n = team.n
    playoff_matrix, draft_matrix = team.payoff_matrix()

    # Expected payoff matrix as weighted average where each scalar is applied to its matrix and the results are summed 
    expected_matrix = n * playoff_matrix + (1 - n) * draft_matrix
    return expected_matrix

In [5]:
# for wins, gp in [(5, 20), (10, 20), (15, 20), (20, 20)]:
#     t = Team("Testers", wins=wins, games_played=gp)
#     print(f" Wins={wins}/{gp} (n={t.n:.3f})  →  Scalar Payoff = {t.describe_matrix():.3f}")

In [6]:
def get_team_game_log(team_id, season):
    """
    Gets a team's regular‐season game log from stats.nba.com API ENDPOINT for a season, returning a DataFrame sorted by date with columns:
    [GAME_DATE, MATCHUP, WL, CUM_WINS, GAMES_PLAYED]
    """
    url = 'https://stats.nba.com/stats/teamgamelog'
    params = {
        'TeamID': team_id,
        'Season': season,
        'SeasonType': 'Regular Season'
    }
    resp = requests.get(url, headers=HEADERS, params=params, timeout=10)
    resp.raise_for_status()

    data = resp.json()['resultSets'][0]
    columns, rows = data['headers'], data['rowSet']

    df = pd.DataFrame(rows, columns=columns)
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format="%b %d, %Y")
    df = df.sort_values('GAME_DATE').reset_index(drop=True)

    # cumulative wins and games played
    wins = 0
    cum_wins = []
    for i, row in df.iterrows():
        if row['WL'] == 'W':
            wins += 1
        cum_wins.append(wins)
    df['CUM_WINS'] = cum_wins
    df['GAMES_PLAYED'] = df.index + 1

    return df


In [ ]:
season = '2023-24'

standings_url = 'https://stats.nba.com/stats/leaguestandingsv3'
params = {
    'LeagueID': '00',
    'Season': season,
    'SeasonType': 'Regular Season'
}
resp = requests.get(standings_url, headers=HEADERS, params=params)
resp.raise_for_status()

data = resp.json()['resultSets'][0]
columns = data['headers']
rows = data['rowSet']
df_standings = pd.DataFrame(rows, columns=columns)

# Extract TeamID and full TeamName
teams_info = [(int(row['TeamID']), row['TeamName']) for _, row in df_standings.iterrows()]

# Static map from 3-letter abbreviation to TeamID:
abbr_map = {
    'ATL': 1610612737, 'BOS': 1610612738, 'BKN': 1610612751, 'CHA': 1610612766,
    'CHI': 1610612741, 'CLE': 1610612739, 'DAL': 1610612742, 'DEN': 1610612743,
    'DET': 1610612765, 'GSW': 1610612744, 'HOU': 1610612745, 'IND': 1610612754,
    'LAC': 1610612746, 'LAL': 1610612747, 'MEM': 1610612763, 'MIA': 1610612748,
    'MIL': 1610612749, 'MIN': 1610612750, 'NOP': 1610612740, 'NYK': 1610612752,
    'OKC': 1610612760, 'ORL': 1610612753, 'PHI': 1610612755, 'PHX': 1610612756,
    'POR': 1610612757, 'SAC': 1610612758, 'SAS': 1610612759, 'TOR': 1610612761,
    'UTA': 1610612762, 'WAS': 1610612764
}


In [ ]:
team_logs = {}
missing = []  # to record any team_ids that fail permanently which was used in API debugging 
# some debug constnats because i had some issues with the API previously 
DELAY_BETWEEN_REQUESTS = 0.2  # seconds between API calls
MAX_RETRIES_429 = 3          # maximum retries on HTTP 429

for team_id, team_name in teams_info:
    attempt = 0
    while True:
        attempt += 1
        try:
            df_log = get_team_game_log(team_id, season)
            team_logs[team_id] = (team_name, df_log)
            break 

        except requests.exceptions.HTTPError as http_err:
            status = http_err.response.status_code
            print(f"HTTP error for {team_name} ({team_id}) in {season}: {http_err}")
            missing.append(team_id)
            break

        except requests.exceptions.Timeout:
            print(f"Timeout fetching {team_name} ({team_id}) for {season} on attempt {attempt}")
            missing.append(team_id)
            break

        except Exception as e:
            print(f"Error fetching {team_name} ({team_id}) for {season}: {e}")
            missing.append(team_id)
            break

    time.sleep(DELAY_BETWEEN_REQUESTS)

print(f"\nFinished building team_logs.  Successful entries: {len(team_logs)}")
if missing:
    print(f"Failed to fetch {len(missing)} teams. Sample: {missing[:5]}")


In [ ]:
max_games = max(len(df) for (_, df) in team_logs.values())

# init DataFrame: index = 1..max_games, columns = team IDs
df_payoffs = pd.DataFrame(index=range(1, max_games + 1), columns=list(team_logs.keys()), dtype=float)

for team_id, (team_name, df_team) in team_logs.items():
    for idx, row in df_team.iterrows():
        t = idx + 1
        wins_self = row['CUM_WINS']
        gp_self = row['GAMES_PLAYED']
        self_n = wins_self / gp_self
        # getting abv name of team from matchup
        opp_abbr = row['MATCHUP'].strip().split()[-1]
        opp_id = abbr_map.get(opp_abbr, None)
        if opp_id is None or opp_id not in team_logs:
            continue

        opp_df = team_logs[opp_id][1]
        if len(opp_df) > idx:
            opp_wins = opp_df.loc[idx, 'CUM_WINS']
            opp_gp = opp_df.loc[idx, 'GAMES_PLAYED']
            opp_n = opp_wins / opp_gp
        else:
            continue

        # If the team's own win-rate < 0.30, strongly bias toward tanking as instructed in the previous work
        if self_n < 0.30:
            n_eff = 0.05  # Very low coefficient - almost unconditional tank
        else:
            # else, compute blended coefficient
            n_eff = (self_n + (1 - opp_n)) / 2

        team_obj = Team(team_name, wins_self, gp_self)
        M_exp = team_obj.expected_payoff(n_eff)
        scalar_payoff = M_exp.sum()

        df_payoffs.at[t, team_id] = scalar_payoff

In [ ]:
plt.figure(figsize=(12, 6))
for team_id in df_payoffs.columns:
    team_name = team_logs[team_id][0]
    plt.plot(
        df_payoffs.index,
        df_payoffs[team_id].values,
        label=team_name,
        linewidth=1
    )

plt.title(f"Scalar Payoff Over Time for All Teams ({season})\n(Adjusted by Opponent + Tank Threshold)")
plt.xlabel("Game Number")
plt.ylabel("Scalar Payoff (sum of $M_{\\mathrm{exp}}$ entries)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()


In [ ]:
def normalize_columnwise(df):
    col_min = df.min()
    col_max = df.max()
    denom = (col_max - col_min).replace(0, 1)
    return (df - col_min) / denom

df_payoffs_norm = normalize_columnwise(df_payoffs)

plt.figure(figsize=(12, 6))
for team_id in df_payoffs_norm.columns:
    team_name = team_logs[team_id][0]
    plt.plot(
        df_payoffs_norm.index,
        df_payoffs_norm[team_id].values,
        label=team_name,
        alpha=0.7,
        linewidth=1
    )

plt.title(f"Normalized Scalar Payoff Over Time for All Teams ({season})\n(Adjusted by Opponent + Tank Threshold)")
plt.xlabel("Game Number")
plt.ylabel("Normalized Scalar Payoff (0–1)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

In [ ]:

original_dict = abbr_map
reversed_dict = {value: key for key, value in original_dict.items()}
print(reversed_dict) 

In [ ]:
for i in range(len(df_payoffs_norm.columns)):
    print(i, reversed_dict[df_payoffs_norm.columns[i]])
# print(reversed_dict[df_payoffs_norm.columns])

In [ ]:
# example team - the pistons, who sucked really bad 
plt.figure(figsize=(12, 6))
team_id = df_payoffs_norm.columns[28]
team_name = team_logs[team_id][0]
plt.plot(
    df_payoffs_norm.index,
    df_payoffs_norm[team_id].values,
    label=team_name,
    alpha=0.7,
    linewidth=1
)

plt.title(f"Normalized Scalar Payoff Over Time for All Teams ({season})\n(Adjusted by Opponent)")
plt.xlabel("Game Number")
plt.ylabel("Normalized Scalar Payoff (0–1)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

In [ ]:
avg_norm_payoff = df_payoffs_norm.mean(axis=1)

# 5 game sliding window mean to decrease variance across games 
avg_norm_smoothed = avg_norm_payoff.rolling(window=5, min_periods=1, center=True).mean()

plt.figure(figsize=(10, 4))
plt.plot(
    avg_norm_smoothed.index,
    avg_norm_smoothed.values,
    color='darkorange',
    linewidth=2
)
plt.title(f"Smoothed League‐Average Normalized Payoff (5‐Game Window): {season}\n(Adjusted by Opponent + Tank Threshold)")
plt.xlabel("Game Number")
plt.ylabel("Smoothed Avg Normalized Payoff (0–1)")
plt.tight_layout()
plt.show()